In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import math
from sklearn import mixture
from sklearn.utils import shuffle
from skimage import measure
from glob import glob
import os
from multiprocessing import Pool, cpu_count
from functools import partial
from subprocess import check_output

In [2]:
print(check_output(["ls", "../train"]).decode("utf-8"))

Type_1
Type_2
Type_3



In [3]:
TRAIN_DATA = "../train"

In [4]:
types = ['Type_1', 'Type_2', 'Type_3']

In [5]:
type_ids = []

In [6]:
%%time
for type in enumerate(types):
    type_i_files = glob(os.path.join(TRAIN_DATA, type[1], "*.jpg"))
    print(type_i_files[:1])
    type_i_ids = np.array([s[len(TRAIN_DATA)+8:-4] for s in type_i_files])
    print(type_i_ids[:1])
    type_ids.append(type_i_ids)

['../train/Type_1/579.jpg']
['579']
['../train/Type_2/992.jpg']
['992']
['../train/Type_3/835.jpg']
['835']
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 8.82 ms


In [7]:
def get_filename(image_id, image_type):
    """
    Method to grab image file path from its id and tpye
    """
    if image_type == "Type_1" or \
        image_type == "Type_2" or \
        image_type == "Type_3":
        data_path = os.path.join(TRAIN_DATA, image_type)
    else:
        raise Exception("Image type {} is not recognized".format(image_type))
        
    ext = 'jpg'
    return os.path.join(data_path, "{}.{}".format(image_id, ext))

In [8]:
def get_image_data(image_id, image_type):
    """
    Method to get image data as np.array that specifies image id and type
    """
    fname = get_filename(image_id, image_type)
    img = cv2.imread(fname)
    assert img is not None, "Failed to read image: {}, {}".format(image_id, image_type)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [9]:
# Crop images that may have circular frames present
# Find the largest inscribed rectangled in the thresholded image
# image is cropped to that rectangle

def maxHist(hist):
    maxArea = (0, 0, 0)
    height = []
    position = []
    for i in range(len(hist)):
        if (len(height) == 0):
            if (hist[i] > 0):
                height.append(hist[i])
                position.append(i)
        else: 
            if (hist[i] > height[-1]):
                height.append(hist[i])
                position.append(i)
            elif (hist[i] < height[-1]):
                while (height[-1] > hist[i]):
                    maxHeight = height.pop()
                    area = maxHeight * (i-position[-1])
                    if (area > maxArea[0]):
                        maxArea = (area, position[-1], i)
                    last_position = position.pop()
                    if (len(height) == 0):
                        break
                position.append(last_position)
                if (len(height) == 0):
                    height.append(hist[i])
                elif(height[-1] < hist[i]):
                    height.append(hist[i])
                else:
                    position.pop()    
    while (len(height) > 0):
        maxHeight = height.pop()
        last_position = position.pop()
        area =  maxHeight * (len(hist) - last_position)
        if (area > maxArea[0]):
            maxArea = (area, len(hist), last_position)
    return maxArea

In [10]:
def maxRect(img):
    maxArea = (0, 0, 0)
    addMat = np.zeros(img.shape)
    for r in range(img.shape[0]):
        if r == 0:
            addMat[r] = img[r]
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
        else:
            addMat[r] = img[r] + addMat[r-1]
            addMat[r][img[r] == 0] *= 0
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
    return (int(maxArea[3]+1-maxArea[0]/abs(maxArea[1]-maxArea[2])), maxArea[2], maxArea[3], maxArea[1], maxArea[0])

In [11]:
def cropCircle(img):
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    img = cv2.resize(img, dsize=tile_size)
            
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY);
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

    _, contours, _ = cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    main_contour = sorted(contours, key = cv2.contourArea, reverse = True)[0]
            
    ff = np.zeros((gray.shape[0],gray.shape[1]), 'uint8') 
    cv2.drawContours(ff, main_contour, -1, 1, 15)
    ff_mask = np.zeros((gray.shape[0]+2,gray.shape[1]+2), 'uint8')
    cv2.floodFill(ff, ff_mask, (int(gray.shape[1]/2), int(gray.shape[0]/2)), 1)
    
    rect = maxRect(ff)
    rectangle = [min(rect[0],rect[2]), max(rect[0],rect[2]), min(rect[1],rect[3]), max(rect[1],rect[3])]
    img_crop = img[rectangle[0]:rectangle[1], rectangle[2]:rectangle[3]]
    cv2.rectangle(ff,(min(rect[1],rect[3]),min(rect[0],rect[2])),(max(rect[1],rect[3]),max(rect[0],rect[2])),3,2)
    
    return [img_crop, rectangle, tile_size]

In [12]:
# For initial delineation of cervix, use two features
    # a color: the higher the a value, the redder the pixel color
    # R: distance of a pixel from image center -> spatial information
    #    -> support extraction of continuous regions

def Ra_space(img, Ra_ratio, a_threshold):
    imgLab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB);
    w = img.shape[0]
    h = img.shape[1]
    Ra = np.zeros((w*h, 2))
    for i in range(w):
        for j in range(h):
            R = math.sqrt((w/2-i)*(w/2-i) + (h/2-j)*(h/2-j))
            Ra[i*h+j, 0] = R
            Ra[i*h+j, 1] = min(imgLab[i][j][1], a_threshold)
            
    Ra[:,0] /= max(Ra[:,0])
    Ra[:,0] *= Ra_ratio
    Ra[:,1] /= max(Ra[:,1])

    return Ra

In [13]:
# image is separated into two clusters in the 2-D (a-R) features space
# use of Gaussian mixture modeling
# initialized by k-means procedure

def get_and_crop_image(image_id, image_type):
    img = get_image_data(image_id, image_type)
    initial_shape = img.shape
    [img, rectangle_cropCircle, tile_size] = cropCircle(img)
    imgLab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB);
    w = img.shape[0]
    h = img.shape[1]
    Ra = Ra_space(imgLab, 1.0, 150)
    a_channel = np.reshape(Ra[:,1], (w,h))
    
    g = mixture.GaussianMixture(n_components = 2, covariance_type = 'diag', random_state = 0, init_params = 'kmeans')
    image_array_sample = shuffle(Ra, random_state=0)[:1000]
    g.fit(image_array_sample)
    labels = g.predict(Ra)
    labels += 1 # Add 1 to avoid labeling as 0 since regionprops ignores the 0-label.
    
    # The cluster that has the highest a-mean is selected.
    labels_2D = np.reshape(labels, (w,h))
    gg_labels_regions = measure.regionprops(labels_2D, intensity_image = a_channel)
    gg_intensity = [prop.mean_intensity for prop in gg_labels_regions]
    cervix_cluster = gg_intensity.index(max(gg_intensity)) + 1

    mask = np.zeros((w * h,1),'uint8')
    mask[labels==cervix_cluster] = 255
    mask_2D = np.reshape(mask, (w,h))

    cc_labels = measure.label(mask_2D, background=0)
    regions = measure.regionprops(cc_labels)
    areas = [prop.area for prop in regions]

    regions_label = [prop.label for prop in regions]
    largestCC_label = regions_label[areas.index(max(areas))]
    mask_largestCC = np.zeros((w,h),'uint8')
    mask_largestCC[cc_labels==largestCC_label] = 255

    img_masked = img.copy()
    img_masked[mask_largestCC==0] = (0,0,0)
    img_masked_gray = cv2.cvtColor(img_masked, cv2.COLOR_RGB2GRAY);
            
    _,thresh_mask = cv2.threshold(img_masked_gray,0,255,0)
            
    kernel = np.ones((11,11), np.uint8)
    thresh_mask = cv2.dilate(thresh_mask, kernel, iterations = 1)
    thresh_mask = cv2.erode(thresh_mask, kernel, iterations = 2)
    _, contours_mask, _ = cv2.findContours(thresh_mask.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    main_contour = sorted(contours_mask, key = cv2.contourArea, reverse = True)[0]
    cv2.drawContours(img, main_contour, -1, 255, 3)
    
    x,y,w,h = cv2.boundingRect(main_contour)
    
    rectangle = [x+rectangle_cropCircle[2],
                 y+rectangle_cropCircle[0],
                 w,
                 h,
                 initial_shape[0],
                 initial_shape[1],
                 tile_size[0],
                 tile_size[1]]

    return [image_id, img, rectangle]

In [14]:
def parallelize_image_cropping(image_ids):
    out = open('rectangles.csv', "w")
    out.write("image_id,type,x,y,w,h,img_shp_0_init,img_shape1_init,img_shp_0,img_shp_1\n")
    imf_d = {}
    p = Pool(cpu_count())
    for type in enumerate(types):
        partial_get_and_crop = partial(get_and_crop_image, image_type = type[1])    
        ret = p.map(partial_get_and_crop, image_ids[type[0]])
        for i in range(len(ret)):
            out.write(image_ids[type[0]][i])
            out.write(',' + str(type[1]))
            out.write(',' + str(ret[i][2][0]))
            out.write(',' + str(ret[i][2][1]))
            out.write(',' + str(ret[i][2][2]))
            out.write(',' + str(ret[i][2][3]))
            out.write(',' + str(ret[i][2][4]))
            out.write(',' + str(ret[i][2][5]))
            out.write(',' + str(ret[i][2][6]))
            out.write(',' + str(ret[i][2][7]))
            out.write('\n')
            img = get_image_data(image_ids[type[0]][i], type[1])
            if(img.shape[0] > img.shape[1]):
                tile_size = (int(img.shape[1]*256/img.shape[0]), 256)
            else:
                tile_size = (256, int(img.shape[0]*256/img.shape[1]))
            img = cv2.resize(img, dsize=tile_size)
            cv2.rectangle(img,
                          (ret[i][2][0], ret[i][2][1]), 
                          (ret[i][2][0]+ret[i][2][2], ret[i][2][1]+ret[i][2][3]),
                          255,
                          2)
            #plt.imshow(img)
            #plt.show()
        ret = []
    out.close()

    return

In [15]:
%%time
parallelize_image_cropping(type_ids)

CPU times: user 39min 46s, sys: 1min 33s, total: 41min 20s
Wall time: 11min 38s


## Import Segmented Images

In [16]:
df = pd.read_csv('rectangles.csv')

In [17]:
df.head(5)

,image_id,type,x,y,w,h,img_shp_0_init,img_shape1_init,img_shp_0,img_shp_1
0,579,Type_1,1,60,190,155,4128,3096,192,256
1,787,Type_1,5,36,173,183,3264,2448,192,256
2,660,Type_1,1,33,170,179,3264,2448,192,256
3,102,Type_1,26,47,136,160,3264,2448,192,256
4,1226,Type_1,1,71,155,124,4128,3096,192,256


In [18]:
df.shape

(1480, 10)

In [19]:
all_files = []

for type in enumerate(types):
    all_files.append(glob(os.path.join(TRAIN_DATA, type[1], "*.jpg")))

all_files = np.concatenate(all_files)

In [20]:
item_dict = {}
for item in all_files:
    item_idx = int(item.split('/')[-1][:-4])
    
    match = df[df['image_id'] == item_idx]
    
    xd = int(match['x'].item())
    yd = int(match['y'].item())
    wd = int(match['w'].item())
    hd = int(match['h'].item())
    img_shp_0d = int(match['img_shp_0'].item())
    img_shp_1d = int(match['img_shp_1'].item())
    type_d = match['type'].item()
    
    item_dict[item_idx] = (xd, yd, wd, hd, img_shp_0d, img_shp_1d, type_d) 
    # xd:0, yd:1, wd:2, hd:3, img_0d:4, img_1d:5

In [21]:
%%time

labels = []
segmented_images = []

for item in all_files:
    # grab image id from filename
    item_ind = int(item.split('/')[-1][:-4])
    img = cv2.imread(item)
    
    # return resizing information from dict
    x_i, y_i, w_i, h_i, img_shp_0_i, img_shp_1_i, type_i = item_dict[item_ind]
    
    labels.append(type_i)
    
    # resize to new image dimensions
    resized = cv2.resize(img, (img_shp_0_i, img_shp_1_i))

    # crop image to segmented area
    crop_img = resized[y_i:y_i+h_i, x_i:x_i+w_i]
    
    # resize image to an even smaller value for faster processing
    desired_size = 64
    old_size = crop_img.shape[:2]
    
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    
    im = cv2.resize(crop_img, (new_size[1], new_size[0]))
    
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
    value=color)
    
    segmented_images.append(new_im)

CPU times: user 4min 57s, sys: 12.9 s, total: 5min 10s
Wall time: 5min 10s


In [22]:
len(labels), len(segmented_images)

(1480, 1480)

In [23]:
from collections import Counter
Counter(labels)

Counter({'Type_1': 249, 'Type_2': 781, 'Type_3': 450})

## Building a Simple CNN

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [26]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(labels)

LabelEncoder()

In [27]:
le.classes_

array(['Type_1', 'Type_2', 'Type_3'], 
      dtype='<U6')

In [28]:
labels = le.transform(labels)

In [29]:
Counter(labels)

Counter({0: 249, 1: 781, 2: 450})

In [30]:
X_train, X_test, y_train, y_test = train_test_split(segmented_images, 
                                                    labels, 
                                                    test_size=0.25, 
                                                    random_state=42,
                                                   stratify=labels)

In [31]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [32]:
X_train = X_train.reshape(X_train.shape[0], 64, 64, 3)
X_test = X_test.reshape(X_test.shape[0], 64, 64, 3)

In [33]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train = X_train / 255.0
X_test = X_test / 255.0

In [34]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train, num_classes=3)
y_test = np_utils.to_categorical(y_test, num_classes=3)
num_classes = y_test.shape[1]

In [35]:
X_test.shape, X_train.shape

((370, 64, 64, 3), (1110, 64, 64, 3))

In [36]:
y_test.shape, y_train.shape

((370, 3), (1110, 3))

In [37]:
def conv_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3),
                     data_format="channels_last"))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [38]:
model = conv_model()

# Fit the model
# make sure batch size is perfectly divisible
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=20,
verbose=2)

Train on 1110 samples, validate on 370 samples
Epoch 1/10
13s - loss: 1.2723 - acc: 0.4973 - val_loss: 0.9795 - val_acc: 0.5270
Epoch 2/10
12s - loss: 0.9929 - acc: 0.5225 - val_loss: 1.0047 - val_acc: 0.5216
Epoch 3/10
12s - loss: 0.9738 - acc: 0.5351 - val_loss: 1.0077 - val_acc: 0.5135
Epoch 4/10
12s - loss: 0.9247 - acc: 0.5495 - val_loss: 1.0196 - val_acc: 0.5270
Epoch 5/10
12s - loss: 0.8865 - acc: 0.5793 - val_loss: 0.9974 - val_acc: 0.4946
Epoch 6/10
12s - loss: 0.8601 - acc: 0.5847 - val_loss: 1.0178 - val_acc: 0.5000
Epoch 7/10
12s - loss: 0.8373 - acc: 0.6108 - val_loss: 1.0883 - val_acc: 0.5108
Epoch 8/10
12s - loss: 0.7830 - acc: 0.6351 - val_loss: 1.0839 - val_acc: 0.5081
Epoch 9/10
12s - loss: 0.7482 - acc: 0.6640 - val_loss: 1.1345 - val_acc: 0.4973
Epoch 10/10
12s - loss: 0.6966 - acc: 0.7180 - val_loss: 1.1469 - val_acc: 0.4865


## Dense Net

In [174]:
from keras.layers import BatchNormalization, concatenate, Activation, AveragePooling2D, GlobalAveragePooling2D
from keras.regularizers import l2

In [175]:
def relu(x): return Activation('relu')(x)
def dropout(x, p): return Dropout(p)(x) if p else x
def bn(x): return BatchNormalization(axis=-1)(x)
def relu_bn(x): return relu(bn(x))

In [176]:
def conv(x, nf, sz, wd, p):
    x = Convolution2D(filters=nf, kernel_size=sz, strides=(sz, sz), kernel_initializer='he_uniform', padding='same', 
                          kernel_regularizer=l2(wd))(x)
    return dropout(x,p)

In [177]:
def conv_block(x, nf, bottleneck=False, p=None, wd=0):
    x = relu_bn(x)
    if bottleneck: x = relu_bn(conv(x, nf * 4, 1, wd, p))
    return conv(x, nf, 3, wd, p)

In [192]:
def dense_block(x, nb_layers, growth_rate, bottleneck=False, p=None, wd=0):
    if bottleneck: nb_layers //= 2
    for i in range(nb_layers):
        b = conv_block(x, growth_rate, bottleneck=bottleneck, p=p, wd=wd)
        x = concatenate([x,b], axis=-1)
    return x

In [193]:
def transition_block(x, compression=1.0, p=None, wd=0):
    nf = int(x.get_shape().as_list()[-1] * compression)
    x = relu_bn(x)
    x = conv(x, nf, 1, wd, p)
    return AveragePooling2D((2, 2), strides=(2, 2))(x)

In [194]:
def create_dense_net(nb_classes, img_input, depth=40, nb_block=3, 
     growth_rate=12, nb_filter=16, bottleneck=False, compression=1.0, p=None, wd=0, activation='softmax'):
    
    assert activation == 'softmax' or activation == 'sigmoid'
    #assert (depth - 4) % nb_block == 0
    nb_layers_per_block = int((depth - 4) / nb_block)
    nb_layers = [nb_layers_per_block] * nb_block

    x = conv(img_input, nb_filter, 3, wd, 0)
    for i,block in enumerate(nb_layers):
        x = dense_block(x, block, growth_rate, bottleneck=bottleneck, p=p, wd=wd)
        if i != len(nb_layers)-1:
            x = transition_block(x, compression=compression, p=p, wd=wd)

    x = relu_bn(x)
    x = GlobalAveragePooling2D()(x)
    return Dense(nb_classes, activation=activation, kernel_regularizer=l2(wd))(x)

In [195]:
input_shape = (64,64,3)
img_input = Input(shape=input_shape)
x = create_dense_net(3, img_input, depth=12, nb_filter=16, compression=0.5, 
                     bottleneck=True, p=0.2, wd=1e-4)

ValueError: `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 22, 22, 16), (None, 8, 8, 12)]